# Notebook for deploying on aws and testing model

BLABLABALBALBA Lorem Ipsum BALBALABLAB

---

In [1]:
# import aws sagemaker apis
import boto3
import sagemaker
from sagemaker import get_execution_role

In [2]:
# SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# default S3 bucket
bucket = sagemaker_session.default_bucket()

In [3]:
# specify where to upload in S3
data_dir = './data/dataset/train'
prefix = 'dog-breed-data'

s3_keys = [obj.key for obj in boto3.resource('s3').Bucket(bucket).objects.all()]
# upload to S3 - Due to the size of the dataset it takes some time to finish...
if any(prefix in s for s in s3_keys):
    input_data = f"s3://{bucket}/{prefix}"
else:
    input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
print(input_data)

s3://sagemaker-sa-east-1-031843945636/dog-breed-data


## PyTorch Estimator

In [7]:
# import a PyTorch wrapper
from sagemaker.pytorch import PyTorch

# specify an output path
# prefix is specified above
output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir='source', # this should be just "source" for your code
                    role=role,
                    framework_version='1.0',
                    train_instance_count=1,
                    train_instance_type='ml.c5.xlarge',
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'input_dim': 3,  # num of features
                        'hidden_dim': 256,
                        'output_dim': 119,
                        'epochs': 5, # could change to higher
                        'batch-size': 256
                    })

In [8]:
estimator.fit({'train': input_data})

2020-05-14 00:33:35 Starting - Starting the training job...
2020-05-14 00:33:39 Starting - Launching requested ML instances......
2020-05-14 00:34:43 Starting - Preparing the instances for training...
2020-05-14 00:35:30 Downloading - Downloading input data............
2020-05-14 00:37:22 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-05-14 00:37:38,712 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-05-14 00:37:38,715 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-05-14 00:37:38,724 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-05-14 00:37:40,134 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-05-14 00:37:40,414 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-05-14 00:

UnexpectedStatusException: Error for Training job sagemaker-pytorch-2020-05-14-00-33-35-082: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/usr/bin/python -m train --batch-size 256 --epochs 5 --hidden_dim 256 --input_dim 3 --output_dim 119"
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.torch/models/resnet50-19c8e357.pth
  0%|          | 0/102502400 [00:00<?, ?it/s]  0%|          | 8192/102502400 [00:00<24:20, 70180.06it/s]  0%|          | 40960/102502400 [00:00<18:52, 90464.60it/s]  0%|          | 106496/102502400 [00:00<14:07, 120780.55it/s]  0%|          | 245760/102502400 [00:00<10:18, 165266.76it/s]  1%|          | 524288/102502400 [00:00<07:25, 229146.31it/s]  1%|          | 1064960/102502400 [00:00<05:16, 320487.54it/s]  2%|â         | 2162688/102502400 [00:00<03:42, 451131.36it/s]  4%|â         | 4358144/102502400 [00:00<02:33, 637832.28it/s]  8%|â         | 8404992/102502400 [00:01<01:44, 903917.50it/s] 11

In [ ]:
output_path